# Optimal Power flow

#### Parameter Definitions

In our data file, we have lots of values, and multiple sheets. Take a moment to make sure you understand what every parameter means, conceptually. Below you can find the relevant parameter definitions.

##### Node (i.e. Bus) Data
**Parameter Name** | **Description**
---|---
Vmax        | Maximal voltage of node
Vmin        | Minimal voltage of node
VAnglemax   | Maximal voltage angle of node
VAnglemin   | Minimal voltage angle of node
Pload       | Real power drawn by loads at node
Qload       | Reactive power drawn by loads at node
PGmax       | Maximal real power of generators at node
PGmin       | Minimal real power of generators at node
QGmax       | Maximal reactive power of generators at node
QGmin       | Minimal reactive power of generators at node
a           | Generator cost function first coefficient# a*p^2
b           | Generator cost function second coefficient# b*p
c           | Generator cost function third coefficient# c

where the cost function of a dispatchable generator at node $i$, with a generation power of $p_i$, is defined as

$$ C_i(p_i) = a_i p_i^2 + b_i p_i + c_i $$

subject to the power constraints $ P^{G,min}_i \le p_i \le P^{Gmax}_i $.

**Note**: The slack node, i.e. the reference node, serves as an angular reference for all other nodes in the system, and is thus set to 0 deg. At this slack node, the voltage magnitude is also assumed to be 1.0 p.u.

##### Admittance Matrices

To model the admittance between two buses, i and k, we have set up the following matrices.
One matrix is for the real part of the admittance, while the other is for the imaginary part.
Since these matrices cover all relationships between buses, these are square matrices.
An example for the structure of the Real Admittance Matrix is shown below:

$$\begin{bmatrix}
G_{1,1} & G_{1,2} & \dots & \dots & G_{1,n} \\
G_{2,1} & \ddots & & & \vdots \\
\vdots & & G_{i,k} & & \vdots \\
\vdots & & & \ddots & G_{n-1,n} \\
G_{n,1} & \dots & \dots & G_{n,n-1} & G_{n,n} 
\end{bmatrix}$$

**Knowledge Checkup:**
To make sure you understand the data structure, take a look at the data file(s) and see if you can answer the following conceptual questions:

1) How many buses are there in total?
2) Which buses have generators connected to them?
3) Which buses have loads connected to them?
4) Which one of these nodes is the slack node?

After you have answered these questions for yourself, you're all ready to code!

#### Dependency Setup

In [28]:
import os
import numpy as np
import pandas as pd
import pyomo.environ as pyo
from opf_utils import solve_and_print, create_results_json

# TODO: Your email address (required for NEOS)
os.environ['NEOS_EMAIL'] = 'jihad.jundi@tum.de'
FILENAME = 'OPFData.xlsx'

#### Model Setup

In [29]:
# filename = filename of the data source
# m = model
def add_node_data(filename, m):
    # TODO: Load the pandas dataframes for the node data.
    # Hint: Pay attention to the file types and headers. 
    data = pd.read_excel(filename, sheet_name='NodeData')

    # TODO: Set the number of nodes
    m.nodes = pyo.Set(initialize=data.index.tolist())
    
    # TODO: Complete this function
    m.V_max = pyo.Param(m.nodes, initialize=data['Vmax'].to_dict())
    m.V_min = pyo.Param(m.nodes, initialize=data['Vmin'].to_dict())
    m.V_angle_max = pyo.Param(m.nodes, initialize=data['VAnglemax'].to_dict())
    m.V_angle_min = pyo.Param(m.nodes, initialize=data['VAnglemin'].to_dict())
    m.P_load = pyo.Param(m.nodes, initialize=data['Pload'].to_dict())
    m.Q_load = pyo.Param(m.nodes, initialize=data['Qload'].to_dict())
    m.P_G_max = pyo.Param(m.nodes, initialize=data['PGmax'].to_dict())
    m.P_G_min = pyo.Param(m.nodes, initialize=data['PGmin'].to_dict())
    m.Q_G_max = pyo.Param(m.nodes, initialize=data['QGmax'].to_dict())
    m.Q_G_min = pyo.Param(m.nodes, initialize=data['QGmin'].to_dict())
    m.a = pyo.Param(m.nodes, initialize=data['a'].to_dict())
    m.b = pyo.Param(m.nodes, initialize=data['b'].to_dict())
    m.c = pyo.Param(m.nodes, initialize=data['c'].to_dict())

    return m

def add_real_admittance_data(filename, m):
    # TODO: Load the pandas dataframes for the node data.
    # Hint: Pay attention to the file types and headers.
    data = pd.read_excel(filename, sheet_name='RealAdmittanceMatrix', header=None)
    assert data.shape[0] == data.shape[1], "The admittance data is not a square matrix"

    # TODO: Complete this function
    data_dict = {(i, j): data.iloc[i, j] for i in range(data.shape[0]) for j in range(data.shape[1])}
    m.G = pyo.Param(m.nodes, m.nodes, initialize=data_dict)
    return m

def add_imag_admittance_data(filename, m):
    # TODO: Load the pandas dataframes for the node data.
    # Hint: Pay attention to the file types and headers.
    data = pd.read_excel(filename, sheet_name='ImaginaryAdmittanceMatrix', header=None)
    assert data.shape[0] == data.shape[1], "The admittance data is not a square matrix"

    # TODO: Complete this function
    data_dict = {(i, j): data.iloc[i, j] for i in range(data.shape[0]) for j in range(data.shape[1])}
    m.B = pyo.Param(m.nodes, m.nodes, initialize=data_dict)
    return m

#### AC Optimal Power Flow Setup

In [30]:
def objective_rule(m):
    # Objective function to minimize the cost of generation
    return sum(m.a[i] * m.x_p[i]**2 + m.b[i] * m.x_p[i] + m.c[i] for i in m.nodes)

# m = model
# i = node index
def power_bounds_rule(m, i):
    # TODO: Complete this function
    return (m.P_G_min[i], m.x_p[i], m.P_G_max[i]) # Eq. 1.4

def voltage_angle_bounds_rule(m, i):
    # TODO: Complete this function
    return (m.V_angle_min[i], m.x_v_angle[i], m.V_angle_max[i]) # Eq. 1.7

def reactive_power_bounds_rule(m, i):
    # TODO: Complete this function
    return (m.Q_G_min[i], m.x_q[i], m.Q_G_max[i]) # Eq. 1.5

def voltage_bounds_rule(m, i):
    # TODO: Complete this function
    return (m.V_min[i], m.x_v[i], m.V_max[i]) # Eq. 1.6

def power_balance_rule_ac(m, i):
    # TODO: Complete this function
    return m.x_p[i] - m.P_load[i] - sum(
        m.x_v[i] * m.x_v[k] * (m.G[i, k] * pyo.cos(m.x_v_angle[i] - m.x_v_angle[k]) + m.B[i, k] * pyo.sin(m.x_v_angle[i] - m.x_v_angle[k]))
        for k in m.nodes if (i, k) in m.G) == 0

def reactive_power_balance_rule(m, i):
    # TODO: Complete this function
    return m.x_q[i] - m.Q_load[i] - sum(
        m.x_v[i] * m.x_v[k] * (m.G[i, k] * pyo.sin(m.x_v_angle[i] - m.x_v_angle[k]) - m.B[i, k] * pyo.cos(m.x_v_angle[i] - m.x_v_angle[k]))
        for k in m.nodes if (i, k) in m.B) == 0

def add_linear_constraints_ac(m):
    # TODO: Complete this function
    m.power_bounds = pyo.Constraint(m.nodes, rule=power_bounds_rule)
    m.voltage_angle_bounds = pyo.Constraint(m.nodes, rule=voltage_angle_bounds_rule)
    m.reactive_power_bounds = pyo.Constraint(m.nodes, rule=reactive_power_bounds_rule)
    m.voltage_bounds = pyo.Constraint(m.nodes, rule=voltage_bounds_rule)
    return m

def add_nonlinear_constraints_ac(m):
    m.power_balance = pyo.Constraint(m.nodes, rule=power_balance_rule_ac)
    m.reactive_power_balance = pyo.Constraint(m.nodes, rule=reactive_power_balance_rule)
    return m

##### DC Optimal Power Flow Setup

In [31]:
# m = model
# i = node index
def power_balance_rule_dc(m, i):
    # TODO: Complete this function, similar to power_balance_rule_ac
    return m.x_p[i] - m.P_load[i] - sum(
        m.B[i, k] * (m.x_v_angle[i] - m.x_v_angle[k])
        for k in m.nodes) == 0  # Eq. 1.12

def add_linear_constraints_dc(m):
    # TODO: Complete this function, similar to add_linear_constraints_ac
    m.power_bounds = pyo.Constraint(m.nodes, rule=lambda m, i: (m.P_G_min[i], m.x_p[i], m.P_G_max[i]))  # Eq. 1.13
    m.voltage_angle_bounds = pyo.Constraint(m.nodes, rule=lambda m, i: (m.V_angle_min[i], m.x_v_angle[i], m.V_angle_max[i]))  # Eq. 1.14
    return m

def add_nonlinear_constraints_dc(m):
    # TODO: Complete this function
    m.power_balance = pyo.Constraint(m.nodes, rule=power_balance_rule_dc)
    return m

##### Load Model from Data

In [32]:
# Loads the data from the given file into the model, defines the optimization problem, and returns the model at the end.
def load_model(filename, m, AC=True):
    # Fill the model with the parameters from the files
    m = add_node_data(filename, m)
    m = add_real_admittance_data(filename, m)
    m = add_imag_admittance_data(filename, m)

    # Define optimization variables
    m.x_p = pyo.Var(m.nodes,
                    within=pyo.Reals,
                    doc='Optimization variable (power generation)')
    m.x_q = pyo.Var(m.nodes,
                    within=pyo.Reals,
                    doc='Optimization variable (reactive power generation)')
    m.x_v = pyo.Var(m.nodes,
                    within=pyo.Reals,
                    doc='Optimization variable (voltage)')
    m.x_v_angle = pyo.Var(m.nodes,
                         within=pyo.Reals,
                         doc='Optimization variable (voltage angle)')
    
    # Define objective function
    m.objective_function = pyo.Objective(
        rule=objective_rule,
        sense=pyo.minimize,
        doc='minimize(cost = sum of all generator operating costs)')
    
    # Define the linear constraints
    if AC:
        m = add_linear_constraints_ac(m)
    else:
        m = add_linear_constraints_dc(m)
    
    # Define the nonlinear constraints
    if AC:
        m = add_nonlinear_constraints_ac(m)
    else:
        m = add_nonlinear_constraints_dc(m)

    return m

##### Solve AC Model

In [33]:
AC = True
model = load_model(FILENAME, pyo.ConcreteModel(), AC)
solve_and_print(model, AC)
# TODO: Uncomment if you want to write the result to json file
team_name = 'Group_E' # Your team name, ex. 'Group_A'
create_results_json(model, team_name, AC)

model.name="unknown";
    - termination condition: optimal
    - message from solver: CONOPT 3.17A\x3a Locally optimal; objective
      0.40327670067527355; 16 iterations; evals\x3a nf = 16, ng = 10, nc = 39,
      nJ = 13, nH = 0, nHv = 2



Solution


Node		Voltage angle [deg]
1 		 0.0
2 		 -0.14796059165524722
3 		 -0.06511621768860057
4 		 -0.1622079660115808
5 		 -0.15284194528448056

Node		Voltage magnitude
1 		 1.0
2 		 0.9839069187561671
3 		 0.9652306188370666
4 		 0.9710647027309575
5 		 0.95

Node		Power
1 		 0.9451009559504059
2 		 0.0
3 		 0.1940926357042062
4 		 0.056662469172039515
5 		 0.0

Node		Reactive Power
1 		 0.6099494009675075
2 		 0.0
3 		 -0.08980595470082486
4 		 0.2
5 		 0.0

Cost:  0.40327670067527355
Results written to Group_E_AC.json


##### Solve DC Model

In [34]:
AC = False
model = load_model(FILENAME, pyo.ConcreteModel(), AC)
solve_and_print(model, AC)
# TODO: Uncomment if you want to write the result to json file
team_name = 'Group_E' # Your team name, ex. 'Group_A'
create_results_json(model, team_name, AC)




Solution


Node		Voltage angle [deg]
1 		 0.0
2 		 -0.1414435796344642
3 		 -0.06395090506031664
4 		 -0.15704500792803033
5 		 -0.1458754148359298

Node		Power
1 		 0.9013967112387036
2 		 0.0
3 		 0.18750000911161477
4 		 0.05010327964968249
5 		 0.0

Cost:  0.3833375053333433
Results written to Group_E_DC.json
